In [1]:
# Characterisation can be done by changing settings to qibolab/runcards/tiiq.yml and diagnostics.yml
# These scripts do not save the characterisation results on the runcard; to do so use 
#   ds.backup_config_file()
#   resonator_freq, avg_min_voltage, max_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy()
#   ds.save_config_parameter('resonator_freq', resonator_freq, 'characterization', 'single_qubit', qubit)
import pathlib
import os
from qibolab.paths import qibolab_folder
from qibolab import Platform
from qibolab.calibration import utils
# from qibolab.calibration.calibration import Calibration as Diagnostics

script_folder = pathlib.Path(os.path.abspath(''))
# diagnostics_settings = script_folder / 'examples' / 'tii' / "diagnostics.yml"
diagnostics_settings = script_folder / "diagnostics.yml"
runcard = qibolab_folder / 'runcards' / 'qw5q.yml' 

# Create a platform; connect and configure it
platform = Platform('multiqubit', runcard)
platform.connect()
platform.setup()

import pathlib
from scipy.signal import savgol_filter
from qibolab.paths import qibolab_folder
import numpy as np
import matplotlib.pyplot as plt
import yaml

from quantify_core.measurement import MeasurementControl
from quantify_core.measurement.control import Gettable, Settable
from quantify_core.data.handling import set_datadir

from qibolab import Platform
from qibolab.paths import qibolab_folder
from qibolab.calibration import utils
from qibolab.calibration import fitting
from qibolab.pulses import Pulse, ReadoutPulse, Rectangular, Gaussian, Drag
from qibolab.circuit import PulseSequence


class Diagnostics():

    def __init__(self, platform: Platform, settings_file = None,  show_plots=True):
        self.platform = platform
        if not settings_file:
            script_folder = pathlib.Path(__file__).parent
            settings_file = script_folder / "diagnostics.yml"
        self.settings_file = settings_file
        # TODO: Set mc plotting to false when auto calibrates (default = True for diagnostics)
        self.mc, self.pl, self.ins = utils.create_measurement_control('Calibration', show_plots)

    def load_settings(self):
        # Load calibration settings
        with open(self.settings_file, "r") as file:
            self.settings = yaml.safe_load(file)
            self.software_averages = self.settings['software_averages']
            self.software_averages_precision = self.settings['software_averages_precision']
            self.max_num_plots = self.settings['max_num_plots']

    def reload_settings(self):
        self.load_settings()

    #--------------------------#
    # Single qubit experiments #
    #--------------------------#

    def run_resonator_spectroscopy(self, qubit=0):
        platform = self.platform
        platform.reload_settings()
        mc = self.mc

        sequence = PulseSequence()
        ro_pulse = platform.qubit_readout_pulse(qubit, start = 0)
        sequence.add(ro_pulse)

        self.reload_settings()
        self.lowres_width = self.settings['resonator_spectroscopy']['lowres_width']
        self.lowres_step = self.settings['resonator_spectroscopy']['lowres_step']
        self.highres_width = self.settings['resonator_spectroscopy']['highres_width']
        self.highres_step = self.settings['resonator_spectroscopy']['highres_step']
        self.precision_width = self.settings['resonator_spectroscopy']['precision_width']
        self.precision_step = self.settings['resonator_spectroscopy']['precision_step']

        self.pl.tuids_max_num(self.max_num_plots)
        platform.qrm[qubit].lo.frequency = platform.characterization['single_qubit'][qubit]['resonator_freq'] - ro_pulse.frequency
        #Fast Sweep
        if (self.software_averages !=0):
            scanrange = utils.variable_resolution_scanrange(self.lowres_width, self.lowres_step, self.highres_width, self.highres_step)
            mc.settables(platform.qrm[qubit].lo.settable_frequency)
            mc.setpoints(scanrange + platform.qrm[qubit].lo.frequency)
            mc.gettables(Gettable(ROController(platform, sequence, qubit)))
            platform.start() 
            dataset = mc.run("Resonator Spectroscopy Fast", soft_avg=self.software_averages)
            platform.stop()
            platform.qrm[qubit].lo.frequency = dataset['x0'].values[dataset['y0'].argmin().values]
            avg_max_voltage = np.mean(dataset['y0'].values[:(self.lowres_width//self.lowres_step)]) * 1e6

        # Precision Sweep
        if (self.software_averages_precision !=0):
            scanrange = np.arange(-self.precision_width, self.precision_width, self.precision_step)
            mc.settables(platform.qrm[qubit].lo.settable_frequency)
            mc.setpoints(scanrange + platform.qrm[qubit].lo.frequency)
            mc.gettables(Gettable(ROController(platform, sequence, qubit)))
            platform.start()
            dataset = mc.run("Resonator Spectroscopy Precision", soft_avg=self.software_averages_precision)
            platform.stop()

        # Fitting
        smooth_dataset = savgol_filter(dataset['y0'].values, 25, 2)
        # resonator_freq = dataset['x0'].values[smooth_dataset.argmin()] + ro_pulse.frequency
        min_ro_voltage = smooth_dataset.min() * 1e6

        f0, BW, Q = fitting.lorentzian_fit("last", min, "Resonator_spectroscopy")
        resonator_freq = (f0*1e9 + ro_pulse.frequency)

        print(f"\nResonator Frequency = {resonator_freq}")
        return resonator_freq, avg_max_voltage, min_ro_voltage, smooth_dataset, dataset


    def run_qubit_spectroscopy(self, qubit=0):
        platform = self.platform
        platform.reload_settings()
        mc = self.mc

        sequence = PulseSequence()
        qd_pulse = platform.qubit_drive_pulse(qubit, start = 0, duration = 5000) 
        ro_pulse = platform.qubit_readout_pulse(qubit, start = 5000)
        sequence.add(qd_pulse)
        sequence.add(ro_pulse)

        self.reload_settings()
        self.fast_start = self.settings['qubit_spectroscopy']['fast_start']
        self.fast_end = self.settings['qubit_spectroscopy']['fast_end']
        self.fast_step = self.settings['qubit_spectroscopy']['fast_step']
        self.precision_start = self.settings['qubit_spectroscopy']['precision_start']
        self.precision_end = self.settings['qubit_spectroscopy']['precision_end']
        self.precision_step = self.settings['qubit_spectroscopy']['precision_step']

        self.pl.tuids_max_num(self.max_num_plots)
        platform.qrm[qubit].lo.frequency = platform.characterization['single_qubit'][qubit]['resonator_freq'] - ro_pulse.frequency
        
        # Fast Sweep
        if (self.software_averages !=0):
            platform.qcm[qubit].lo.frequency = platform.characterization['single_qubit'][qubit]['qubit_freq'] + qd_pulse.frequency
            lo_qcm_frequency = platform.qcm[qubit].lo.frequency
            fast_sweep_scan_range = np.arange(self.fast_start, self.fast_end, self.fast_step)
            mc.settables(platform.qcm[qubit].lo.settable_frequency)
            mc.setpoints(fast_sweep_scan_range + lo_qcm_frequency)
            mc.gettables(Gettable(ROController(platform, sequence, qubit)))
            platform.start() 
            dataset = mc.run("Qubit Spectroscopy Fast", soft_avg=self.software_averages)
            platform.stop()
            platform.qcm[qubit].lo.frequency = dataset['x0'].values[dataset['y0'].argmax().values]
            avg_min_voltage = np.mean(dataset['y0'].values[:((self.fast_end - self.fast_start)//self.lowres_step)]) * 1e6

        # Precision Sweep
        if (self.software_averages_precision !=0):
            lo_qcm_frequency = platform.qcm[qubit].lo.frequency
            precision_sweep_scan_range = np.arange(self.precision_start, self.precision_end, self.precision_step)
            mc.settables(platform.qcm[qubit].lo.settable_frequency)
            mc.setpoints(precision_sweep_scan_range + lo_qcm_frequency)
            mc.gettables(Gettable(ROController(platform, sequence, qubit)))
            platform.start() 
            dataset = mc.run("Qubit Spectroscopy Precision", soft_avg=self.software_averages_precision)
            platform.stop()

        # Fitting
        smooth_dataset = savgol_filter(dataset['y0'].values, 11, 2)
        qubit_freq = dataset['x0'].values[smooth_dataset.argmax()] - qd_pulse.frequency
        max_ro_voltage = smooth_dataset.max() * 1e6

        print(f"\nQubit Frequency = {qubit_freq}")
        utils.plot(smooth_dataset, dataset, "Qubit_Spectroscopy", 1)
        print("Qubit freq ontained from MC results: ", qubit_freq)
        f0, BW, Q = fitting.lorentzian_fit("last", max, "Qubit_Spectroscopy")
        qubit_freq = (f0*1e9 - qd_pulse.frequency)
        print("Qubit freq ontained from fitting: ", qubit_freq)
        return qubit_freq, max_ro_voltage, smooth_dataset, dataset



        
class ROController():
    # Quantify Gettable Interface Implementation
    label = ['Amplitude', 'Phase','I','Q']
    unit = ['V', 'Radians','V','V']
    name = ['A', 'Phi','I','Q']

    def __init__(self, platform, sequence, qubit):
        self.platform = platform
        self.sequence = sequence
        self.qubit = qubit

    def get(self):
        results = self.platform.execute_pulse_sequence(self.sequence)
        return list(list(results.values())[0].values())[0] #TODO: Replace with the particular acquisition



# create a diagnostics/calibration object
ds = Diagnostics(platform, diagnostics_settings)

qubit = 0


[Qibo 0.1.7|INFO|2022-06-23 07:39:56]: Using numpy backend on /CPU:0
[Qibo 0.1.7|WARNING|2022-06-23 07:39:56]: numpy backend uses `np.einsum` and supports CPU only. Consider installing the qibojit or qibotf backends for increased performance and to enable GPU acceleration.
[Qibo 0.1.7|INFO|2022-06-23 07:39:57]: Loading platform multiqubit from runcard d:\alvaro\projects\github\qibolab\src\qibolab\runcards\qw5q.yml
[Qibo 0.1.7|INFO|2022-06-23 07:39:57]: Connecting to multiqubit instrument qrm_rf.
[Qibo 0.1.7|INFO|2022-06-23 07:39:58]: Connecting to multiqubit instrument qcm_rf1.
[Qibo 0.1.7|INFO|2022-06-23 07:39:58]: Connecting to multiqubit instrument qcm_rf2.
[Qibo 0.1.7|INFO|2022-06-23 07:39:58]: Connecting to multiqubit instrument qcm_rf3.
[Qibo 0.1.7|INFO|2022-06-23 07:40:08]: Creating MeasurementControl Calibration


In [ ]:
for qubit in range(5):
    resonator_freq, avg_max_voltage, min_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy(qubit)

for qubit in range(5):
    qubit_freq, max_ro_voltage, smooth_dataset, dataset = ds.run_qubit_spectroscopy(qubit)

In [ ]:
resonator_freq, avg_min_voltage, max_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy(qubit)


In [2]:
qubit_freq, max_ro_voltage, smooth_dataset, dataset = ds.run_qubit_spectroscopy(qubit)

Starting iterative measurement...
100% completed | elapsed time:    222s | time left:      0s  
100% completed | elapsed time:    222s | time left:      0s  

AttributeError: 'Diagnostics' object has no attribute 'lowres_width'